# Tutorial 7: Graph Neural Networks

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Under%20development&color=red)

**Filled notebook:** 
[![View on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/GNN_overview.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/GNN_overview.ipynb)  


**Pre-trained models:** 

Explanation/Introduction...

In [1]:
## Standard libraries
import os
import json
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install pytorch-lightning==1.0.3
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/tutorial7"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
print(device)

cuda:0


## GNN models

### Graph representation

* Adjacency matrix
* List of edges

While expressing a graph as a list of edges is more efficient, using an adjacency matrix is more intuitive and simpler to implement. We will use the list of edges to define a sparse adjacency matrix with which we can work as if it was a dense matrix, but allows more efficient operations.

* Introduction to `torch.sparse` and its concepts

### Graph Convolutions

In [2]:
class GCNLayer(nn.Module):
    
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)
        
    
    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats

In [3]:
layer = GCNLayer(2, 2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
node_feats = torch.arange(6, dtype=torch.float32).view(1, 3, 2)
adj_matrix = torch.Tensor([[[1, 1, 0],
                            [0, 1, 0],
                            [0, 0, 1]]])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Adjacency matrix tensor([[[1., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.]]])
Output features tensor([[[1., 2.],
         [2., 3.],
         [4., 5.]]])


### Graph Attention 

In [4]:
class GATLayer(nn.Module):
    
    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Number of output features must be a multiple of the count of heads."
            c_out = c_out // num_heads
        
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out))
        self.leakyrelu = nn.LeakyReLU(alpha)
        
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)
        
        edges = adj_matrix.nonzero(as_tuple=False)

        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * batch_size + edges[:,1]
        edge_indices_col = edges[:,0] * batch_size + edges[:,2]
        
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1)
        
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a) 
        attn_logits = self.leakyrelu(attn_logits)
        
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)
        
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", attn_probs.permute(0, 3, 1, 2))
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)
        
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)
        
        return node_feats 

In [5]:
layer = GATLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.15, -0.1]])
node_feats = torch.arange(6, dtype=torch.float32).view(1, 3, 2)
adj_matrix = torch.Tensor([[[1, 1, 0],
                            [0, 1, 0],
                            [0, 0, 1]]])
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Attention probs
 tensor([[[[0.3543, 0.6457, 0.0000],
          [0.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 1.0000]],

         [[0.5200, 0.4800, 0.0000],
          [0.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 1.0000]]]])
Adjacency matrix tensor([[[1., 1., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.]]])
Output features tensor([[[1.2913, 1.9600],
         [2.0000, 3.0000],
         [4.0000, 5.0000]]])


## PyTorch geometric

In [6]:
# torch geometric
try: 
    import torch_geometric
except ModuleNotFoundError:
    !pip install torch_geometric
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

RDKit WARNING: [17:07:06] Enabling RDKit 2019.09.3 jupyter extensions


In [7]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

GraphConv:

$$
\mathbf{x}_v^{(\ell+1)} = \mathbf{W}^{(\ell + 1)}_1 \mathbf{x}_v^{(\ell)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{w \in \mathcal{N}(v)} \mathbf{x}_w^{(\ell)}
$$

## Graph Tasks

### Node-level tasks: Semi-supervised node classification

In [8]:
cora_dataset = torch_geometric.datasets.Planetoid(root=DATASET_PATH, name="Cora")

In [9]:
cora_dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [33]:
class GNNModel(nn.Module):
    
    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]
        
        layers = []
        for l_idx in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=c_in if l_idx == 0 else c_hidden, 
                          out_channels=c_hidden,
                          **kwargs),
                nn.ReLU(),
                nn.Dropout(dp_rate)
            ]
        layers += [gnn_layer(in_channels=c_hidden, 
                             out_channels=c_out,
                             **kwargs)]
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x, edge_index):
        for l in self.layers:
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                x = l(x)
        return x

In [34]:
class NodeLevelGNN(pl.LightningModule):
    
    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()
        
        self.model = GNNModel(**model_kwargs)
        self.loss_module = nn.CrossEntropyLoss()
        
    
    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)
        
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, "Unknown forward mode: %s" % mode
        
        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return loss, acc
        
        
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer
        
        
    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss
        
        
    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)
        
        
    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

In [35]:
def train_node_classifier(model_name, dataset, **model_kwargs):
    pl.seed_everything(42)
    node_data_loader = geom_data.DataLoader(dataset, batch_size=1)
    
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "NodeLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         checkpoint_callback=ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=200,
                         progress_bar_refresh_rate=0) # 0 because epoch size is 1
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "NodeLevel%s.ckpt" % model_name)
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = NodeLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        model = NodeLevelGNN(c_in=dataset.num_node_features, c_out=dataset.num_classes, **model_kwargs)
        trainer.fit(model, node_data_loader, node_data_loader)
    # Test best model on validation and test set
    test_result = trainer.test(model, test_dataloaders=node_data_loader, verbose=False)
    result = {"test": test_result}
    return model, result

In [76]:
model, result = train_node_classifier(model_name="GCN", dataset=cora_dataset, c_hidden=16, layer_name="GCN")

GPU available: True, used: True
I1109 17:16:49.605558 140454539228992 distributed.py:49] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
I1109 17:16:49.607161 140454539228992 distributed.py:49] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I1109 17:16:49.608504 140454539228992 accelerator_connector.py:385] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: ../saved_models/tutorial7/NodeLevelGraphConv/lightning_logs
W1109 17:16:49.643353 140454539228992 tensorboard.py:241] Missing logger folder: ../saved_models/tutorial7/NodeLevelGraphConv/lightning_logs

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | GNNModel         | 200 K 
1 | loss_module | CrossEntropyLoss | 0     
I1109 17:16:49.647999 140454539228992 lightning.py:1290] 
  | Name        | Type             | Params
-------------------------------------------------
0 | model       | GNNModel   

Exception ignored in: <function tqdm.__del__ at 0x7fbd992310d0>
Traceback (most recent call last):
  File "/home/phillip/anaconda3/envs/nlp1/lib/python3.7/site-packages/tqdm/std.py", line 1087, in __del__
    self.close()
  File "/home/phillip/anaconda3/envs/nlp1/lib/python3.7/site-packages/tqdm/notebook.py", line 249, in close
    self.sp(close=True)
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


In [14]:
print(result)

{'test': [{'train_loss': 0.031641338020563126, 'train_acc': 1.0, 'val_acc': 0.7839999794960022, 'test_acc': 0.8240000009536743}]}


### Edge-level tasks: Link prediction

### Graph-level tasks: Graph classification

https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=2owRWKcuoALo

<center width="100%"><img src="torch_geometric_stacking_graphs.png" width="600px"></center>

In [55]:
class GraphGNNModel(nn.Module):
    
    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        super().__init__()
        self.GNN = GNNModel(c_in=c_in, 
                            c_hidden=c_hidden, 
                            c_out=c_hidden*2, # Not our prediction output yet!
                            **kwargs)
        self.head = nn.Sequential(
            nn.Dropout(dp_rate_linear),
            nn.Linear(c_hidden*2, c_out)
        )
        
    
    def forward(self, x, edge_index, batch_idx):
        x = self.GNN(x, edge_index)
        x = geom_nn.global_max_pool(x, batch_idx)
        x = self.head(x)
        return x

In [148]:
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="Tox21_MMP_training")

Extracting ../data/Tox21_MMP_training/Tox21_MMP_training.zip
Processing...
Done!


In [149]:
len(tu_dataset)

7320

In [150]:
tu_dataset[0].y

tensor([1])

In [151]:
torch.cat([tu_dataset[i].y for i in range(len(tu_dataset))]).float().mean()

tensor(0.1560)

In [133]:
graph_train_loader = geom_data.DataLoader(tu_dataset, batch_size=16, shuffle=True)
graph_val_loader = geom_data.DataLoader(tu_dataset, batch_size=16)
graph_test_loader = geom_data.DataLoader(tu_dataset, batch_size=16)

In [134]:
batch = next(iter(graph_train_loader))
print(batch)
print(batch.y)

Batch(batch=[264], edge_attr=[540, 3], edge_index=[2, 540], x=[264, 38], y=[16])
tensor([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])


In [135]:
class GraphLevelGNN(pl.LightningModule):
    
    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()
        
        self.model = GraphGNNModel(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()
        
    
    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)
        
        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc
        
        
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1e-4)
        return optimizer
        
        
    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss
        
        
    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)
        
        
    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

In [136]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)
    
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         checkpoint_callback=ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=10,
                         progress_bar_refresh_rate=1)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "GraphLevel%s.ckpt" % model_name)
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        model = GraphLevelGNN(c_in=tu_dataset.num_node_features, 
                              c_out=1 if tu_dataset.num_classes==2 else tu_dataset.num_classes, 
                              **model_kwargs)
        trainer.fit(model, graph_train_loader, graph_val_loader)
    # Test best model on validation and test set
    test_result = trainer.test(model, test_dataloaders=graph_test_loader, verbose=False)
    result = {"test": test_result}
    return model, result

In [137]:
model, result = train_graph_classifier(model_name="GraphConv", c_hidden=64, layer_name="GAT", num_layers=4, dp_rate_linear=0.0)

GPU available: True, used: True
I1109 17:31:35.921088 140454539228992 distributed.py:49] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
I1109 17:31:35.923042 140454539228992 distributed.py:49] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I1109 17:31:35.924565 140454539228992 accelerator_connector.py:385] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | model       | GraphGNNModel     | 37 K  
1 | loss_module | BCEWithLogitsLoss | 0     
I1109 17:31:35.958180 140454539228992 lightning.py:1290] 
  | Name        | Type              | Params
--------------------------------------------------
0 | model       | GraphGNNModel     | 37 K  
1 | loss_module | BCEWithLogitsLoss | 0     


In [139]:
result

{'test': [{'train_loss': 0.22024297714233398,
   'train_acc': 0.875,
   'val_acc': 0.800000011920929,
   'test_acc': 0.800000011920929}]}

**==========================================**

**==========================================**
## Old code

#### Sparse version

As there doesn't exist a batch-wise matrix multiplication function in the sparse package yet, we have to do it ourselves:

In [ ]:
def bspmm(mat1, mat2):
    return torch.stack([torch.spmm(mat1[i], mat2[i]) for i in range(mat1.shape[0])], dim=0)

In [ ]:
class GCNSparseLayer(GCNLayer):
    
    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Sparse Tensor with entries of 1 where there is an edge between nodes (including self-connection). 
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Determine number of neighbours for averaging (torch.sparse.sum is same as .sum on sparse matrices)
        num_neighbours = torch.sparse.sum(adj_matrix, dim=-1)
        num_neighbours = num_neighbours.to_dense() # Convert sparse tensor to dense one
        
        node_feats = self.projection(node_feats)
        
        # torch.bmm on sparse matrices
        node_feats = bspmm(adj_matrix, node_feats)
        
        node_feats = node_feats / num_neighbours.unsqueeze(dim=-1)
        return node_feats

In [ ]:
layer = GCNSparseLayer(2, 2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
node_feats = torch.arange(6, dtype=torch.float32).view(1, 3, 2)

adj_matrix = torch.sparse.FloatTensor(torch.LongTensor([[0,0,0], [0,0,1], [0,1,1], [0,2,2]]).t(),
                                      torch.FloatTensor([1, 1, 1, 1]),
                                      torch.Size([1,3,3]))

out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

#### Testing efficiency

We can test the efficiency/gain we have gotten when using the efficient version.

In [ ]:
def test_efficiency(layer_class, layer_sparse_class, c_in, batch_size=4, graph_size=5000, link_prob=0.01, **kwargs):
    normal_layer = layer_class(c_in=c_in, **kwargs)
    sparse_layer = layer_sparse_class(c_in=c_in, **kwargs)
    sparse_layer.load_state_dict(normal_layer.state_dict())
    
    node_feats = torch.randn(batch_size, graph_size, c_in)
    adj_matrix = (torch.rand(batch_size, graph_size, graph_size) < link_prob).float()
    edges = adj_matrix.nonzero().t()
    print(edges.shape)
    sparse_adj_matrix = torch.sparse.FloatTensor(edges,
                                                 torch.ones(edges.shape[1],),
                                                 adj_matrix.shape)
    
    test_layer(normal_layer, node_feats, adj_matrix)
    test_layer(sparse_layer, node_feats, sparse_adj_matrix)
    
    
def test_layer(layer_module, node_feats, adj_matrix):
    torch.cuda.empty_cache() 
    layer_module.to(device)
    node_feats, adj_matrix = node_feats.to(device), adj_matrix.to(device)
    start_time = time.time()
    out = layer_module(node_feats, adj_matrix)
    out.sum().backward()
    end_time = time.time()
    print("Time: %6.5fs" % (end_time - start_time))
    if torch.cuda.is_available():
        print("CUDA memory: %4.2fGB" % (torch.cuda.memory_allocated(device)/1e9))
    
    layer_module.to(torch.device("cpu"))
    del node_feats
    del adj_matrix
    del out

In [ ]:
test_efficiency(GCNLayer, GCNSparseLayer, c_in=512, c_out=512, batch_size=64)

#### Sparse version

In [ ]:
class GATSparseLayer(GATLayer):
    
    def forward(self, node_feats, adj_matrix):
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)
        edges = adj_matrix._indices().t()
        
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * batch_size + edges[:,1]
        edge_indices_col = edges[:,0] * batch_size + edges[:,2]
        
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1)
        
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a) # Shape: [batch, num_nodes, num_nodes, num_heads]
        attn_logits = self.leakyrelu(attn_logits) 
        attn_unprobs = attn_logits.exp()
        
        node_feats_per_head = []
        for h in range(self.num_heads): # Split over heads
            attn_matrix = torch.sparse.FloatTensor(adj_matrix._indices(),
                                                   attn_unprobs[:,h],
                                                   size=adj_matrix.shape)
            node_head_feats = bspmm(attn_matrix, node_feats[:,:,h])
            attn_row_sum = torch.sparse.sum(attn_matrix, dim=-1).to_dense()
            node_head_feats = node_head_feats / attn_row_sum.unsqueeze(dim=-1)
            node_feats_per_head.append(node_head_feats)
        
        if self.concat_heads:
            node_feats = torch.cat(node_feats_per_head, dim=-1)
        else:
            node_feats = sum(node_feats_per_head) / self.num_heads
        
        return node_feats 

In [ ]:
layer = GATSparseLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.15, -0.1]])
node_feats = torch.arange(6, dtype=torch.float32).view(1, 3, 2)

adj_matrix = torch.sparse.FloatTensor(torch.LongTensor([[0,0,0], [0,0,1], [0,1,1], [0,2,2]]).t(),
                                      torch.FloatTensor([1, 1, 1, 1]),
                                      torch.Size([1,3,3]))

out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

In [ ]:
test_efficiency(GATLayer2, GATSparseLayer, c_in=128, c_out=128, num_heads=4, batch_size=4, graph_size=500)